In [2]:
import os
import soundfile as sf  # To handle FLAC properly
import keras
from tensorflow.keras.models import load_model
import tensorflow as tf
#print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

import librosa
import numpy as np
import pandas as pd

2025-02-06 21:43:34.404331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738899814.421697 3881432 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738899814.427015 3881432 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-06 21:43:34.445389: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model = keras.models.load_model("./AVSSpoof/audio_spoofing_model.keras")

model.summary()

W0000 00:00:1738899819.745750 3881432 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 200)            │        25,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │        25,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 179,549 (701.37 KB)

 Trainable params: 59,849 (233.79 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 119,700 (467.58 KB)

In [4]:
# SHOULD HAVE A REUSABLE FUNCTION FOR THIS ALREADY
def audio_mfcc(file_path, n_mfcc=128):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    #print(mfcc.shape)
    return np.mean(mfcc.T, axis=0)  # Mean aggregation over time

In [7]:
def predict_audio_label(model, X):
    """
    Predicts whether an audio sample is 'Bonafide' or 'Spoofed' using a trained model.
    
    Parameters:
        model: Trained machine learning model (supports predict or predict_proba).
        X: MFCC Preprocessed feature array of shape (1, 128).
            - USE 'X = X_train.reshape(1,-1)' TO CHANGE MFCC RESULT SHAPE
    
    Returns:
        predicted_label (str): "Bonafide" or "Spoofed".
        confidence_score (float): Probability confidence score of the prediction.
    """
    # Generate prediction
    prediction = model.predict(X)  # Ensure X.shape = (1, 128)

    # Check if model supports predict_proba (for probabilistic models)
    if hasattr(model, "predict_proba"):
        prediction_proba = model.predict_proba(X)[:, 1]  # Extract probability of class 1 (Bonafide)
    else:
        prediction_proba = prediction  # Direct output (e.g., Neural Network sigmoid output)
    print("Prediction Probability:",prediction_proba)    

    # Convert probability output to a single float
    confidence_score = prediction_proba[0].item()

    # Convert to binary classification 
    binary_prediction = 1 if prediction_proba[0] >= 1 else 0
    print("Binary Prediction:",binary_prediction)
    
    # Convert numerical label to human-readable label
    predicted_label = "Spoofed" if binary_prediction == 0 else "Bonafide"

    # Print results
    #print(f"\n🔹 **Single Prediction Result:**")
    #print(f"Prediction: {predicted_label} (Confidence: {confidence_score:.4f})")

    return predicted_label, confidence_score

In [27]:
#audio_features = "./REAL AUDIO/lijo_Zebra.wav"
audio_features = "./AVSSpoof/LA/LA/ASVspoof2019_LA_eval/flac/LA_E_8844552.flac" 


X_train = audio_mfcc(audio_features)
X = X_train.reshape(1,-1)
#print(X.shape)

# Example usage
predicted_label, confidence_score = predict_audio_label(model, X)

# You can now reference confidence_score separately
print("Your Voice is Considered:", predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction Probability: [[1.]]
Binary Prediction: 1
Your Voice is Considered: Bonafide
